# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-14 22:07:04] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=34421, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=384912776, constrai

[2025-07-14 22:07:16] Attention backend not set. Use fa3 backend by default.
[2025-07-14 22:07:16] Init torch distributed begin.


[2025-07-14 22:07:17] Init torch distributed ends. mem usage=0.00 GB


[2025-07-14 22:07:18] Load weight begin. avail mem=78.50 GB


[2025-07-14 22:07:18] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-14 22:07:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=62.81 GB


[2025-07-14 22:07:22] INFO:     Started server process [3843207]
[2025-07-14 22:07:22] INFO:     Waiting for application startup.
[2025-07-14 22:07:22] INFO:     Application startup complete.
[2025-07-14 22:07:22] INFO:     Uvicorn running on http://0.0.0.0:34421 (Press CTRL+C to quit)


[2025-07-14 22:07:23] INFO:     127.0.0.1:41130 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-14 22:07:23] INFO:     127.0.0.1:41132 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-14 22:07:23] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:23.613859


[2025-07-14 22:07:24] INFO:     127.0.0.1:41148 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:24] The server is fired up and ready to roll!


Server started on http://localhost:34421


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-14 22:07:28] INFO:     127.0.0.1:41162 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-14 22:07:28] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:28.528692


[2025-07-14 22:07:28] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.93, #queue-req: 0, timestamp: 2025-07-14T22:07:28.865843
[2025-07-14 22:07:28] INFO:     127.0.0.1:41174 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-14 22:07:28] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:28.894730


[2025-07-14 22:07:29] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: False, gen throughput (token/s): 102.14, #queue-req: 0, timestamp: 2025-07-14T22:07:29.257461
[2025-07-14 22:07:29] INFO:     127.0.0.1:54522 - "POST /generate HTTP/1.1" 200 OK


[2025-07-14 22:07:29] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 73, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:29.260926


[2025-07-14 22:07:29] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 108.91, #queue-req: 0, timestamp: 2025-07-14T22:07:29.624727
[2025-07-14 22:07:29] INFO:     127.0.0.1:54532 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-14 22:07:29] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:29.634075
[2025-07-14 22:07:29] INFO:     127.0.0.1:54548 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:29] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:29.659793
[2025-07-14 22:07:29] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-14T22:07:29.660857
[2025-07-14 22:07:29] INFO:     127.0.0.1:54562 - "POST /generate HTTP/1.1" 200 OK


[2025-07-14 22:07:29] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:29.695836


[2025-07-14 22:07:30] INFO:     127.0.0.1:54576 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-14 22:07:30] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:30.025161
[2025-07-14 22:07:30] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-14T22:07:30.026416
[2025-07-14 22:07:30] Decode batch. #running-req: 2, #token: 66, token usage: 0.00, cuda graph: False, gen throughput (token/s): 79.59, #queue-req: 0, timestamp: 2025-07-14T22:07:30.190155


[2025-07-14 22:07:30] Decode batch. #running-req: 2, #token: 146, token usage: 0.01, cuda graph: False, gen throughput (token/s): 212.52, #queue-req: 0, timestamp: 2025-07-14T22:07:30.566600


[2025-07-14 22:07:30] Decode batch. #running-req: 2, #token: 226, token usage: 0.01, cuda graph: False, gen throughput (token/s): 203.39, #queue-req: 0, timestamp: 2025-07-14T22:07:30.959940


[2025-07-14 22:07:31] Decode batch. #running-req: 2, #token: 306, token usage: 0.01, cuda graph: False, gen throughput (token/s): 211.24, #queue-req: 0, timestamp: 2025-07-14T22:07:31.338687
[2025-07-14 22:07:31] INFO:     127.0.0.1:54580 - "POST /generate HTTP/1.1" 200 OK


[2025-07-14 22:07:31] INFO:     127.0.0.1:54592 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:31] Prefill batch. #new-seq: 1, #new-token: 317, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:31.622922
[2025-07-14 22:07:31] Decode batch. #running-req: 1, #token: 366, token usage: 0.02, cuda graph: False, gen throughput (token/s): 120.65, #queue-req: 0, timestamp: 2025-07-14T22:07:31.744804


...

[2025-07-14 22:07:32] Decode batch. #running-req: 1, #token: 486, token usage: 0.02, cuda graph: False, gen throughput (token/s): 97.08, #queue-req: 0, timestamp: 2025-07-14T22:07:32.905331


[2025-07-14 22:07:33] Decode batch. #running-req: 1, #token: 526, token usage: 0.03, cuda graph: False, gen throughput (token/s): 66.46, #queue-req: 0, timestamp: 2025-07-14T22:07:33.507166


[2025-07-14 22:07:34] Decode batch. #running-req: 1, #token: 566, token usage: 0.03, cuda graph: False, gen throughput (token/s): 66.65, #queue-req: 0, timestamp: 2025-07-14T22:07:34.107308


[2025-07-14 22:07:34] Decode batch. #running-req: 1, #token: 606, token usage: 0.03, cuda graph: False, gen throughput (token/s): 66.60, #queue-req: 0, timestamp: 2025-07-14T22:07:34.707878


[2025-07-14 22:07:35] INFO:     127.0.0.1:54602 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-14 22:07:35] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:35.161146


[2025-07-14 22:07:37] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, cuda graph: False, gen throughput (token/s): 13.17, #queue-req: 0, timestamp: 2025-07-14T22:07:37.745806
[2025-07-14 22:07:37] INFO:     127.0.0.1:54616 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-14 22:07:37] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:37.987974


[2025-07-14 22:07:38] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, cuda graph: False, gen throughput (token/s): 50.06, #queue-req: 0, timestamp: 2025-07-14T22:07:38.544883


[2025-07-14 22:07:39] Decode batch. #running-req: 1, #token: 114, token usage: 0.01, cuda graph: False, gen throughput (token/s): 68.17, #queue-req: 0, timestamp: 2025-07-14T22:07:39.131637


[2025-07-14 22:07:39] Decode batch. #running-req: 1, #token: 154, token usage: 0.01, cuda graph: False, gen throughput (token/s): 67.76, #queue-req: 0, timestamp: 2025-07-14T22:07:39.721984
[2025-07-14 22:07:39] INFO:     127.0.0.1:54628 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-14 22:07:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:39.765117
[2025-07-14 22:07:39] INFO:     127.0.0.1:46932 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-07-14 22:07:39] Prefill batch. #new-seq: 3, #new-token: 29, #cached-token: 51, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:39.800409


 33%|███▎      | 1/3 [00:00<00:00,  6.99it/s]

100%|██████████| 3/3 [00:00<00:00, 17.44it/s]

[2025-07-14 22:07:39] INFO:     127.0.0.1:46940 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:39] INFO:     127.0.0.1:46962 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:39] INFO:     127.0.0.1:46948 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-14 22:07:39] INFO:     127.0.0.1:46978 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:07:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:39.976652
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-14 22:07:40] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:07:40.116216


[2025-07-14 22:07:40] INFO:     127.0.0.1:46988 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-14 22:07:40] Child process unexpectedly failed with exitcode=9. pid=3843960


[2025-07-14 22:07:40] Child process unexpectedly failed with exitcode=9. pid=3843664


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-14 22:07:51] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=39005, nccl_port=None, mem_fraction_static=0.7866, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=50344126,

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-14 22:07:53] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-14 22:07:54] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-14 22:08:02] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-14 22:08:03] Attention backend not set. Use flashinfer backend by default.
[2025-07-14 22:08:03] Init torch distributed begin.
[2025-07-14 22:08:03] Init torch distributed ends. mem usage=0.00 GB


...

[2025-07-14 22:08:10] INFO:     Started server process [3846557]
[2025-07-14 22:08:10] INFO:     Waiting for application startup.
[2025-07-14 22:08:10] INFO:     Application startup complete.
[2025-07-14 22:08:10] INFO:     Uvicorn running on http://0.0.0.0:39005 (Press CTRL+C to quit)


[2025-07-14 22:08:11] INFO:     127.0.0.1:44416 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-14 22:08:11] INFO:     127.0.0.1:44418 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-14 22:08:11] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:08:11.789150


[2025-07-14 22:08:12] INFO:     127.0.0.1:44428 - "POST /generate HTTP/1.1" 200 OK
[2025-07-14 22:08:12] The server is fired up and ready to roll!


Server started on http://localhost:39005


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-14 22:08:16] INFO:     127.0.0.1:44442 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-14 22:08:18] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-14T22:08:18.060098


[2025-07-14 22:08:18] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 4.66, #queue-req: 0, timestamp: 2025-07-14T22:08:18.862940


[2025-07-14 22:08:19] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 62.15, #queue-req: 0, timestamp: 2025-07-14T22:08:19.506490


[2025-07-14 22:08:19] INFO:     127.0.0.1:44448 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-07-14 22:08:19] Child process unexpectedly failed with exitcode=9. pid=3846861
